In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("C:\\Users\\laimi\\Downloads\\pendingpositions_20240816_043004.csv")

In [ ]:
# Check for missing values
print(df.isnull().sum())
print(df.describe())
print(df.info())

In [ ]:
# clean step

# number of rows before clean
print(f"number of rows before clean: {len(df)}")

df_cleaned = df.dropna(subset="StopPrice").copy()

# convert rows to proper types
df_cleaned["Created"] = pd.to_datetime(df_cleaned["Created"])
df_cleaned["Closed"] = pd.to_datetime(df_cleaned["Closed"])
df_cleaned["PositionSize"] = df_cleaned['NumberOfShares'] * df_cleaned["Bid"]
df_cleaned["StopPricePct"] = ((df_cleaned["StopPrice"] - df_cleaned["Bid"])/df_cleaned["Bid"])*100

# split into longs and shorts
longs = df_cleaned[df_cleaned["NumberOfShares"] > 0].copy()
shorts = df_cleaned[df_cleaned["NumberOfShares"] < 0].copy()

longs_purchased = longs[longs["Purchased"]==1]

print(f"number of rows after clean: {len(df_cleaned)}")
print(f"number of longs: {len(longs)}")
print(f"number of purchased longs: {len(longs_purchased)}")
print(f"number of shorts: {len(shorts)}")

In [ ]:
print("NOTE: the next set of analysis is on long positions")

total_positions = len(longs)
print(f"Total positions: {total_positions}")

num_days = (longs['Created'].max() - longs['Created'].min()).days
print(f"Spanning {num_days} Number of days")

purchased_positions = longs['Purchased'].sum()
purchased_percentage = (purchased_positions / total_positions) * 100
print(f"% Purchased: {purchased_positions} ({purchased_percentage:.2f}%)")

In [ ]:

print(f'Average size of positions: {longs["PositionSize"].mean():.2f}')
print(f'Max position size: {longs["PositionSize"].max():.2f}')
print(f'Min position size: {longs["PositionSize"].min():.2f}')

longs['rolling_average_size'] = longs["PositionSize"].rolling(window=30).mean()

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.lineplot(data=longs, x='Created', y='rolling_average_size')
plt.title('Rolling Average Size of Positions Over Time')
plt.xlabel('Date')
plt.ylabel('Average Size of Positions')
plt.show()

# Plot the distribution of position sizes
plt.figure(figsize=(12, 6))
sns.histplot(longs["PositionSize"], bins=30, kde=True)
plt.title('Distribution of Position Sizes')
plt.xlabel('Position Size')
plt.ylabel('Frequency')
plt.show()

# plot the distribution of purchased position sizes
plt.figure(figsize=(12, 6))
sns.histplot(longs_purchased["PositionSize"], bins=30, kde=True)
plt.title('Distribution of Position Sizes of Purchased Positions')
plt.xlabel('Position Size')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# List the top 10 largest position rows
largest_positions = longs.nlargest(10, ["PositionSize"])
print("\nTop 10 Largest Positions:")
largest_positions[["Ticker","Created","Bid","StopPrice","StopPricePct","Strategy","Purchased","PositionSize"]]

In [ ]:
# List the top 10 largest position rows that were purchased
largest_positions = longs_purchased.nlargest(20, ["PositionSize"])
print("\nTop 10 Largest Positions:")
largest_positions[["Ticker","Created","Bid", "NumberOfShares","StopPrice","StopPricePct","Strategy","Purchased","PositionSize"]]

In [ ]:
smallest_positions = longs.nsmallest(10, ["PositionSize"])
print("\nTop 10 Smallest Positions:")
smallest_positions[["Ticker","Created","Bid","StopPrice","StopPricePct","Strategy","Purchased","PositionSize"]]

In [ ]:
longs['Duration'] = (longs['Closed'] - longs['Created']).dt.days
avg_duration_purchased = longs[longs['Purchased'] == 1]['Duration'].mean()
avg_duration_not_purchased = longs[longs['Purchased'] == 0]['Duration'].mean()
max_duration_purchased = longs[longs['Purchased'] == 1]['Duration'].max()

print(f"Average duration for purchased positions: {avg_duration_purchased:.2f} days")
print(f"Average duration for not purchased positions: {avg_duration_not_purchased:.2f} days")
print(f"Max duration for not purchased positions: {max_duration_purchased:.2f} days")

In [ ]:
avg_stop_loss_all = df['StopPrice'].mean()
avg_stop_loss_purchased = df[df['Purchased'] == 1]['StopPrice'].mean()
avg_stop_loss_not_purchased = df[df['Purchased'] == 0]['StopPrice'].mean()

print(f"Average stop loss for all positions: {avg_stop_loss_all:.2f}")
print(f"Average stop loss for purchased positions: {avg_stop_loss_purchased:.2f}")
print(f"Average stop loss for not purchased positions: {avg_stop_loss_not_purchased:.2f}")